# Argumentation Abstraite (Dung)

**Navigation**: [<- Tweety-4-Belief-Revision](Tweety-4-Belief-Revision.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-6-Structured-Argumentation ->](Tweety-6-Structured-Argumentation.ipynb)

---

## Objectifs pedagogiques

1. Comprendre les cadres d'argumentation abstraits de Dung (AF)
2. Maitriser les semantiques d'acceptabilite (grounded, preferred, stable, complete)
3. Explorer la semantique CF2 pour les graphes avec cycles
4. Generer des frameworks d'argumentation aleatoires

## Prerequis

Executez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

## Duree estimee : 25-30 minutes

> **Note:** La section 4.1.4 (Apprentissage depuis Labellisations) est documentee conceptuellement mais desactivee en raison d'un bug interne Tweety.

In [1]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety + Outils ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration COMPLETE des outils externes ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
    "EPROVER": "",
    "SAT_SOLVER_PYTHON": "",
    "MARCO": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str:
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# --- Auto-detection des outils ---
system = platform.system()
exe_suffix = ".exe" if system == "Windows" else ""

# 1. Clingo (ASP solver) - Tweety attend le REPERTOIRE
for cp in [shutil.which("clingo"), pathlib.Path(f"ext_tools/clingo/clingo{exe_suffix}"),
           pathlib.Path(f"../ext_tools/clingo/clingo{exe_suffix}")]:
    if cp and (isinstance(cp, str) or cp.exists()):
        parent = pathlib.Path(cp).parent if isinstance(cp, str) else cp.parent
        EXTERNAL_TOOLS["CLINGO"] = str(parent.resolve())
        break

# 2. SPASS (Modal logic prover)
for sp in [shutil.which("SPASS"), pathlib.Path(f"ext_tools/spass/SPASS{exe_suffix}"),
           pathlib.Path(f"../ext_tools/spass/SPASS{exe_suffix}")]:
    if sp and (isinstance(sp, str) or sp.exists()):
        EXTERNAL_TOOLS["SPASS"] = str(pathlib.Path(sp).resolve()) if isinstance(sp, pathlib.Path) else sp
        break

# 3. EProver (FOL theorem prover)
for ep in [shutil.which("eprover"), pathlib.Path(f"../ext_tools/EProver/eprover{exe_suffix}"),
           pathlib.Path(f"ext_tools/EProver/eprover{exe_suffix}")]:
    if ep:
        ep_path = pathlib.Path(ep) if isinstance(ep, str) else ep
        if ep_path.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(ep_path.resolve())
            break

# 4. SAT Solver Python (CaDiCaL, Glucose via pySAT)
for sat in [pathlib.Path("../ext_tools/sat_solver.py"), pathlib.Path("ext_tools/sat_solver.py")]:
    if sat.exists():
        EXTERNAL_TOOLS["SAT_SOLVER_PYTHON"] = str(sat.resolve())
        break

# 5. MARCO (MUS enumerator avec Z3)
for mp in [pathlib.Path("../ext_tools/marco.py"), pathlib.Path("ext_tools/marco.py")]:
    if mp.exists():
        EXTERNAL_TOOLS["MARCO"] = str(mp.resolve())
        break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = None
    for jdk_path in [pathlib.Path("jdk-17-portable"), pathlib.Path("../Argument_Analysis/jdk-17-portable")]:
        if jdk_path.exists():
            zulu_dirs = list(jdk_path.glob("zulu*"))
            if zulu_dirs:
                jdk_portable = zulu_dirs[0]
                os.environ["JAVA_HOME"] = str(jdk_portable.resolve())
                print(f"JDK portable: {jdk_portable.name}")
                break

    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("../Argument_Analysis/libs")

        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

# === Resume des outils ===
if jvm_ready:
    print("\n--- Outils disponibles ---")
    for tool, path in EXTERNAL_TOOLS.items():
        if path:
            short_path = path.split(os.sep)[-1] if len(path) > 30 else path
            print(f"  {tool}: {short_path}")
    print(f"\nJVM prete. Outils: {sum(1 for t,p in EXTERNAL_TOOLS.items() if p)}/{len(EXTERNAL_TOOLS)}")

--- Verification JVM Tweety + Outils ---
JDK portable: zulu17.50.19-ca-jdk17.0.11-win_x64
JVM demarree avec 35 JARs.

--- Outils disponibles ---
  CLINGO: clingo
  SPASS: SPASS.exe
  EPROVER: eprover.exe
  SAT_SOLVER_PYTHON: sat_solver.py
  MARCO: marco.py

JVM prete. Outils: 5/5


## Partie 4 : Argumentation Abstraite et Structurée
<a id="partie4"></a>

Nous entrons maintenant au cœur de l'argumentation computationnelle, en commençant par le cadre fondateur de Dung et en progressant vers des approches qui prennent en compte la structure interne des arguments. Cette partie est souvent plus stable car elle repose sur des modules centraux de Tweety.

### 4.1.1 Sémantiques de Dung : Fondements

Les **cadres d'argumentation abstraits** (Dung, 1995) sont définis par :
- Un ensemble d'**arguments** (noeuds)
- Une relation d'**attaque** (arcs dirigés)

Les différentes **sémantiques** déterminent quels ensembles d'arguments sont "acceptables" :

| Sémantique | Définition | Propriétés |
|------------|------------|------------|
| **Conflict-free** | Aucun argument n'attaque un autre dans l'ensemble | Base minimale |
| **Admissible** | Conflict-free + chaque argument se défend | Acceptable |
| **Complete** | Admissible + contient tous les arguments défendus | Fixpoint |
| **Grounded** | Complete minimal (unique) | Sceptique |
| **Preferred** | Complete maximal | Crédule |
| **Stable** | Conflict-free + attaque tout argument externe | Le plus strict |

**Relations d'inclusion** : `Stable ⊆ Preferred ⊆ Complete ⊇ {Grounded}`

In [2]:
# --- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---
print("\n--- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---")

# Vérifier si la JVM est prête
if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple Dung...")
    try:
        # Imports nécessaires pour Dung
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        # Importer plusieurs raisonneurs
        from org.tweetyproject.arg.dung.reasoner import (
            SimpleGroundedReasoner, SimpleStableReasoner, SimplePreferredReasoner,
            SimpleCompleteReasoner, SimpleAdmissibleReasoner, SimpleConflictFreeReasoner
        )
        # Importer Semantics si besoin pour certains raisonneurs plus avancés (non utilisés ici)
        # from org.tweetyproject.arg.dung.semantics import Semantics
        from java.util import Collection # Pour vérifier taille retour

        print("✔️ Imports Dung réussis.")

        # --- Exemple 1: A <-> B -> C ---
        af1 = DungTheory()
        a1 = Argument("a1"); b1 = Argument("b1"); c1 = Argument("c1")
        # Ajouter des arguments
        af1.add(a1); af1.add(b1); af1.add(c1)
        # Ajouter des attaques
        af1.add(Attack(a1, b1)); af1.add(Attack(b1, a1)); af1.add(Attack(b1, c1))

        print("\n--- AF1: a1 <-> b1 -> c1 ---")
        print("Cadre :", af1)

        # Calculer les extensions pour différentes sémantiques
        print("\nCalcul des extensions pour AF1:")
        try:
            # Grounded: getModel() retourne une Extension, getModels() retourne une Collection<Extension>
            grounded_ext = SimpleGroundedReasoner().getModel(af1) # Plus direct pour grounded
            print(f" - Grounded   : {{{grounded_ext}}}") # Afficher comme un ensemble pour la cohérence
        except Exception as e: print(f"   Erreur Grounded: {e}")
        try:
            stable_exts = SimpleStableReasoner().getModels(af1)
            print(f" - Stable     ({stable_exts.size()}):", stable_exts)
        except Exception as e: print(f"   Erreur Stable: {e}")
        try:
            preferred_exts = SimplePreferredReasoner().getModels(af1)
            print(f" - Preferred  ({preferred_exts.size()}):", preferred_exts)
        except Exception as e: print(f"   Erreur Preferred: {e}")
        try:
            complete_exts = SimpleCompleteReasoner().getModels(af1)
            print(f" - Complete   ({complete_exts.size()}):", complete_exts)
        except Exception as e: print(f"   Erreur Complete: {e}")
        try:
            admissible_exts = SimpleAdmissibleReasoner().getModels(af1)
            # Convertir la Collection Java en liste Python pour len()
            # Note: Cela peut être coûteux si la collection est énorme
            admissible_list = list(admissible_exts)
            print(f" - Admissible ({len(admissible_list)}):", admissible_exts) # Afficher la collection Java
        except Exception as e: print(f"   Erreur Admissible: {e}")
        try:
            conflict_free_exts = SimpleConflictFreeReasoner().getModels(af1)
            cf_list = list(conflict_free_exts)
            print(f" - ConflictFree ({len(cf_list)}):", conflict_free_exts) # Attention, peut être très grand
        except Exception as e: print(f"   Erreur ConflictFree: {e}")


        # --- Exemple 2: Cycle a->b->c->a ---
        af_cycle = DungTheory()
        a_cy = Argument("a_cy"); b_cy = Argument("b_cy"); c_cy = Argument("c_cy")
        af_cycle.add(a_cy); af_cycle.add(b_cy); af_cycle.add(c_cy)
        af_cycle.add(Attack(a_cy, b_cy)); af_cycle.add(Attack(b_cy, c_cy)); af_cycle.add(Attack(c_cy, a_cy))

        print("\n--- AF Cycle: a -> b -> c -> a ---")
        print("Cadre :", af_cycle)
        print("\nCalcul des extensions pour AF Cycle:")
        try:
            print(" - Grounded   :", SimpleGroundedReasoner().getModel(af_cycle)) # {}
        except Exception as e: print(f"   Erreur Grounded: {e}")
        try:
            stable_exts_cy = SimpleStableReasoner().getModels(af_cycle)
            print(f" - Stable     ({stable_exts_cy.size()}):", stable_exts_cy) # {}
        except Exception as e: print(f"   Erreur Stable: {e}")
        try:
            preferred_exts_cy = SimplePreferredReasoner().getModels(af_cycle)
            print(f" - Preferred  ({preferred_exts_cy.size()}):", preferred_exts_cy) # [{}]
        except Exception as e: print(f"   Erreur Preferred: {e}")
        try:
            complete_exts_cy = SimpleCompleteReasoner().getModels(af_cycle)
            print(f" - Complete   ({complete_exts_cy.size()}):", complete_exts_cy) # [{}]
        except Exception as e: print(f"   Erreur Complete: {e}")


    except ImportError as e:
        print(f"❌ Erreur d'import pour l'Argumentation Abstraite : {e}")
        print("   Vérifiez le JAR 'org.tweetyproject.arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple Dung: {e_java.message()}")
        # print(e_java.stacktrace()) # Décommenter pour trace Java complète
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple Dung: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---
ℹ️ JVM prête. Exécution de l'exemple Dung...
✔️ Imports Dung réussis.

--- AF1: a1 <-> b1 -> c1 ---
Cadre : <{ a1, c1, b1 },[(b1,c1), (b1,a1), (a1,b1)]>

Calcul des extensions pour AF1:
 - Grounded   : {{}}
 - Stable     (2): [{a1,c1}, {b1}]
 - Preferred  (2): [{a1,c1}, {b1}]
 - Complete   (3): [{a1,c1}, {b1}, {}]
 - Admissible (4): [{a1,c1}, {b1}, {}, {a1}]
 - ConflictFree (5): [{c1}, {a1,c1}, {b1}, {}, {a1}]

--- AF Cycle: a -> b -> c -> a ---
Cadre : <{ a_cy, c_cy, b_cy },[(c_cy,a_cy), (a_cy,b_cy), (b_cy,c_cy)]>

Calcul des extensions pour AF Cycle:
 - Grounded   : {}
 - Stable     (0): []
 - Preferred  (1): [{}]
 - Complete   (1): [{}]


#### Interpretation des extensions Dung

Les resultats ci-dessus illustrent les differences fondamentales entre les semantiques :

**AF1 : a1 <-> b1 -> c1** (attaque mutuelle a1/b1, b1 attaque c1)

| Semantique | Extensions | Interpretation |
|------------|------------|----------------|
| **Grounded** | `{}` | Aucun argument n'est sceptiquement acceptable (conflit symetrique) |
| **Stable** (2) | `{a1,c1}`, `{b1}` | Deux visions du monde mutuellement exclusives |
| **Preferred** (2) | Idem Stable | Extensions maximales = extensions stables ici |
| **Complete** (3) | + `{}` | Ajoute l'extension vide (position agnostique) |
| **Admissible** (4) | + `{a1}` | Tout sous-ensemble d'une preferred est admissible |
| **Conflict-Free** (5) | + `{c1}` | Toute combinaison sans auto-attaque |

**AF Cycle : a -> b -> c -> a** (cycle impair)

| Semantique | Extensions | Interpretation |
|------------|------------|----------------|
| **Grounded** | `{}` | Aucun argument n'est defendable |
| **Stable** (0) | Aucune | Cycle impair = pas d'extension stable |
| **Preferred** (1) | `{}` | Seule l'extension vide est maximale |
| **Complete** (1) | `{}` | Idem |

**Lecons cles :**
1. Les cycles impairs cassent la semantique stable -> utiliser **CF2** (section suivante)
2. **Grounded** est la position la plus prudente (sceptique)
3. **Preferred/Stable** donnent les positions credules (choisir un camp)

### 4.1.2 Sémantique CF2 : Gestion des Cycles

La sémantique **CF2** (Baroni et al., 2005) est conçue pour traiter les cadres avec **cycles impairs**.

**Problème des cycles impairs** :
- Un cycle `a→b→c→a` n'a **pas d'extension stable**
- La sémantique stable échoue sur ces structures

**Solution CF2** :
1. **Décomposition en SCCs** (Strongly Connected Components)
2. **Traitement récursif** des composantes
3. **Propagation** des choix entre composantes

**Avantage** : CF2 garantit toujours au moins une extension, même sur les cadres "pathologiques".

**API Tweety** : `SccCF2Reasoner` implémente cette sémantique.

In [3]:
# --- 4.1.2 Sémantique CF2 ---
print("\n--- 4.1.2 Sémantique CF2 ---")

# Vérifier si la JVM est prête
if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple CF2...")
    try:
        # Imports (peuvent être déjà faits, mais sécurité)
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.reasoner import SccCF2Reasoner
        from org.tweetyproject.arg.dung.semantics import Extension # Pour type hinting éventuel
        from java.util import Collection

        print("✔️ Imports Dung (pour CF2) réussis.")

        # --- Exemple AF2 : Cycle a->b->c->d->e->a, e->f ---
        # (Recréation pour autonomie de la cellule)
        af2 = DungTheory()
        args_cf2_map = {name: Argument(name) for name in "abcdef"} # Utiliser un dictionnaire
        for arg in args_cf2_map.values(): af2.add(arg)

        attacks_cf2 = [("a","b"), ("b","c"), ("c","d"), ("d","e"), ("e","a"), ("e","f")]
        for s, t in attacks_cf2:
             # S'assurer que les arguments existent avant d'ajouter l'attaque
             if s in args_cf2_map and t in args_cf2_map:
                  af2.add(Attack(args_cf2_map[s], args_cf2_map[t]))
             else:
                  print(f"WARN: Argument(s) non trouvé(s) pour l'attaque ({s},{t})")

        print("\n--- AF pour CF2 : Cycle a->b->c->d->e->a, e->f ---")
        print("Cadre :", af2)

        # --- Raisonnement CF2 ---
        cf2_reasoner = SccCF2Reasoner()
        print("\nCalcul des extensions CF2:")
        try:
            cf2_extensions_collection = cf2_reasoner.getModels(af2) # Retourne Collection<Extension>
            if cf2_extensions_collection.isEmpty():
                 print("  (Aucune extension CF2 trouvée)")
            else:
                 # Itérer sur la collection Java
                 ext_iterator = cf2_extensions_collection.iterator()
                 count = 0
                 while ext_iterator.hasNext():
                      ext = ext_iterator.next()
                      # Pour un affichage plus propre des arguments dans l'extension:
                      args_in_ext = ", ".join(sorted([str(arg.getName()) for arg in ext]))
                      print(f"  - {{{args_in_ext}}}")
                      count += 1
                 print(f"  ({count} extension(s) trouvée(s))")

        except jpype.JException as e_cf2_java:
             print(f"  ❌ Erreur Java lors du raisonnement CF2: {e_cf2_java.message()}")
             # print(e_cf2_java.stacktrace())
        except Exception as e_cf2_py:
              print(f"  ❌ Erreur Python lors du raisonnement CF2: {e_cf2_py}")

    # Gestion globale
    except ImportError as e:
        print(f"❌ Erreur d'import pour CF2 : {e}. Vérifiez le JAR 'arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple CF2: {e_java.message()}")
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple CF2: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.1.2 Sémantique CF2 ---
ℹ️ JVM prête. Exécution de l'exemple CF2...
✔️ Imports Dung (pour CF2) réussis.

--- AF pour CF2 : Cycle a->b->c->d->e->a, e->f ---
Cadre : <{ a, b, c, d, e, f },[(b,c), (d,e), (e,a), (a,b), (c,d), (e,f)]>

Calcul des extensions CF2:
  - {b, e}
  - {c, e}
  - {a, c, f}
  - {a, d, f}
  - {b, d, f}
  (5 extension(s) trouvée(s))


#### Interpretation des extensions CF2

La semantique CF2 resout le probleme des cycles impairs en decomposant le graphe :

**Structure du framework :**
- Cycle principal : `a -> b -> c -> d -> e -> a` (cycle de longueur 5 = impair)
- Attaque supplementaire : `e -> f`

**Les 5 extensions trouvees :**

| Extension | Arguments IN | Interpretation |
|-----------|--------------|----------------|
| `{b, e}` | 2 args non adjacents dans le cycle | Un "camp" valide |
| `{c, e}` | 2 args non adjacents | Autre camp valide |
| `{a, c, f}` | `a` et `c` + `f` (protege par `e` absent) | Camp avec `f` |
| `{a, d, f}` | `a` et `d` + `f` | Autre combinaison |
| `{b, d, f}` | `b` et `d` + `f` | Derniere combinaison |

**Pourquoi 5 extensions ?**
- Dans un cycle de 5, on peut choisir 2 arguments non adjacents de 5 facons
- Certaines extensions incluent `f` (quand son attaquant `e` est OUT)

**Avantage de CF2 :**
- Garantit toujours au moins une extension (contrairement a Stable)
- Traite les cycles de facon coherente via decomposition SCC
- Utile pour les debats circulaires ou les preferences cycliques

### 4.1.3 Génération de Cadres Aléatoires

Tweety permet de **générer** des cadres d'argumentation pour :
- **Tests de performance** des raisonneurs
- **Études statistiques** des sémantiques
- **Benchmarks** pour comparer algorithmes

**Paramètres de génération** :
- `numberOfArguments` : Nombre d'arguments à créer
- `attackProbability` : Probabilité d'attaque entre deux arguments (modèle Erdős-Rényi)

**Classes Tweety** :
- `DungTheoryGenerationParameters` : Configuration
- `DefaultDungTheoryGenerator` : Générateur itératif

In [4]:
# --- 4.1.3 Génération de Cadres Dung ---
print("\n--- 4.1.3 Génération de Cadres Dung ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple de génération...")
    try:
        # Imports
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.util import DefaultDungTheoryGenerator, DungTheoryGenerationParameters

        print("✔️ Imports pour génération Dung réussis.")

        # Paramètres de génération
        params = DungTheoryGenerationParameters()
        params.numberOfArguments = 6
        params.attackProbability = 0.25
        # CORRECTION: La ligne suivante est retirée car le champ n'existe probablement plus
        # params.allowSelfAttacks = False
        print(f"ℹ️ Paramètres de génération: {params.numberOfArguments} args, proba_attaque={params.attackProbability} (comportement par défaut pour auto-attaques)")

        generator = DefaultDungTheoryGenerator(params)

        # Générer UN cadre
        generated_af = generator.next()

        print("\nCadre généré aléatoirement :")
        print(f"  Arguments: {generated_af.getNodes()}")
        print(f"  Attaques : {generated_af.getAttacks()}")
        print(f"  Représentation compacte: {generated_af}")

    except ImportError as e:
        print(f"❌ Erreur d'import pour la génération Dung : {e}. Vérifiez le JAR 'arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple génération Dung: {e_java.message()}")
        # print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple génération Dung: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.1.3 Génération de Cadres Dung ---
ℹ️ JVM prête. Exécution de l'exemple de génération...
✔️ Imports pour génération Dung réussis.
ℹ️ Paramètres de génération: 6 args, proba_attaque=0.25 (comportement par défaut pour auto-attaques)

Cadre généré aléatoirement :
  Arguments: <{ a1, a2, a3, a4, a5, a0 },[(a1,a5), (a4,a2), (a2,a5), (a0,a4), (a0,a5), (a3,a1), (a5,a3), (a1,a0)]>
  Attaques : [(a1,a5), (a4,a2), (a2,a5), (a0,a4), (a0,a5), (a3,a1), (a5,a3), (a1,a0)]
  Représentation compacte: <{ a1, a2, a3, a4, a5, a0 },[(a1,a5), (a4,a2), (a2,a5), (a0,a4), (a0,a5), (a3,a1), (a5,a3), (a1,a0)]>


### 4.1.4 Apprentissage de Cadres (Bug Connu)

> ⚠️ **Cette section est documentée mais non exécutable** en raison d'un bug interne Tweety 1.28 (`ClassCastException: Tautology cannot be cast to AssociativePlFormula`).

**Concept pédagogique** :

L'**apprentissage de cadres** vise à reconstruire un AF à partir de **labellisations partielles** (IN, OUT, UNDEC) fournies par un oracle.

**Cas d'usage** :
- **Élicitation de préférences** : Déduire les relations d'attaque depuis des jugements humains
- **Ingénierie de connaissances** : Construire des bases argumentatives depuis des exemples

**Classes Tweety concernées** :
- `SimpleAFLearner` : Apprenant de cadres
- `Entity` : Oracle fournissant les labellisations
- `Semantics` : Énumération (STABLE, PREFERRED, etc.)

In [5]:
# --- 4.1.4 Apprentissage de Cadres Dung (depuis Labellisations) ---
print("\n--- 4.1.4 Apprentissage de Cadres Dung ---")

print("!! SECTION COMMENTÉE !!")
print("L'exécution de cet exemple échoue en raison d'une ClassCastException interne à Tweety")
print("(Tautology cannot be cast to AssociativePlFormula) lors de l'appel à getLabeling/learnLabeling")
print("pour le cadre d'argumentation spécifique utilisé ici. Ceci semble être un bug interne.")

# if not jvm_ready:
#     print("❌ ERREUR: JVM non démarrée.")
# else:
#     print("ℹ️ JVM prête. Exécution de l'exemple d'apprentissage...")
#     try:
#         # Imports
#         import jpype
#         from jpype.types import *
#         from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
#         from org.tweetyproject.arg.dung.learning import SimpleAFLearner
#         from org.tweetyproject.arg.dung.learning.syntax import Entity
#         from org.tweetyproject.arg.dung.semantics import Semantics
#         # Import de Label retiré (correction précédente)
#
#         print("✔️ Imports pour apprentissage Dung réussis.")
#
#         # 1. Définir le cadre "caché"
#         hidden_af = DungTheory()
#         a_learn = Argument("a"); b_learn = Argument("b"); c_learn = Argument("c")
#         hidden_af.add(a_learn); hidden_af.add(b_learn); hidden_af.add(c_learn)
#         hidden_af.add(Attack(a_learn, b_learn)); hidden_af.add(Attack(b_learn, a_learn)); hidden_af.add(Attack(b_learn, c_learn))
#         print("\nCadre caché (à apprendre):", hidden_af)
#
#         # 2. Créer l'Oracle
#         oracle = Entity(hidden_af)
#         arguments_known = oracle.getArguments()
#         print("Arguments connus par l'apprenant:", arguments_known)
#
#         # 3. Créer l'Apprenant
#         learner = SimpleAFLearner(arguments_known)
#         print(f"État initial apprenant: {learner.getNumberOfFrameworks()} cadres compatibles possibles.")
#
#         # 4. Apprendre depuis des labellisations
#         # Labellisation Stable
#         print("\nApprentissage depuis labellisation Stable:")
#         try:
#              labeling_stable = oracle.getLabeling(Semantics.STABLE_SEMANTICS) # C'est ici que l'erreur se produit
#              print(f"  Oracle fournit Labellisation STABLE: {labeling_stable}")
#              learner.learnLabeling(labeling_stable)
#              print(f"  Après STABLE: {learner.getNumberOfFrameworks()} cadres compatibles restants.")
#         except jpype.JException as e_stable:
#               print(f"  Erreur lors de l'obtention/apprentissage de la labellisation Stable: {e_stable.message()}")
#               # Afficher la stacktrace Java peut aider à confirmer l'erreur interne
#               # print(e_stable.stacktrace())
#
#         # Labellisation Conflict-Free (pourrait aussi échouer)
#         print("\nApprentissage depuis labellisation Conflict-Free:")
#         try:
#              labeling_cf = oracle.getLabeling(Semantics.CONFLICTFREE_SEMANTICS)
#              print(f"  Oracle fournit Labellisation CONFLICT_FREE: {labeling_cf}")
#              learner.learnLabeling(labeling_cf)
#              print(f"  Après CONFLICT_FREE: {learner.getNumberOfFrameworks()} cadres compatibles restants.")
#         except jpype.JException as e_cf:
#              print(f"  Erreur lors de l'obtention/apprentissage de la labellisation ConflictFree: {e_cf.message()}")
#
#
#         # 5. Obtenir le(s) cadre(s) appris
#         # Ce code ne sera probablement pas atteint
#         learned_af = learner.getModel()
#         print("\nCadre appris (un des cadres compatibles) :")
#         print(str(learned_af))
#
#     # ... (Blocs except ImportError, JException, Exception comme avant) ...
#     except ImportError as e:
#        print(f"❌ Erreur d'import pour l'apprentissage Dung : {e}.")
#     except jpype.JException as e_java:
#        print(f"❌ Erreur Java générale dans l'exemple apprentissage Dung: {e_java.message()}")
#     except Exception as e_gen:
#        print(f"❌ Erreur Python inattendue dans l'exemple apprentissage Dung: {e_gen}")
#        import traceback
#        traceback.print_exc()


--- 4.1.4 Apprentissage de Cadres Dung ---
!! SECTION COMMENTÉE !!
L'exécution de cet exemple échoue en raison d'une ClassCastException interne à Tweety
(Tautology cannot be cast to AssociativePlFormula) lors de l'appel à getLabeling/learnLabeling
pour le cadre d'argumentation spécifique utilisé ici. Ceci semble être un bug interne.


---

## Resume

Ce notebook a couvert les **cadres d'argumentation abstraits de Dung (1995)** :

| Section | Concepts cles |
|---------|--------------|
| **4.1.1 Bases** | Arguments, attaques, DungTheory, 6 semantiques |
| **4.1.2 CF2** | Decomposition SCC, gestion des cycles impairs |
| **4.1.3 Generation** | DefaultDungTheoryGenerator, tests et benchmarks |
| **4.1.4 Apprentissage** | Concept documente (bug Tweety) |

**Hierarchie des semantiques :**
```
ConflictFree -> Admissible -> Complete -> Grounded (unique, sceptique)
                                      -> Preferred (maximale, credule)
                                      -> Stable (attaque tout, pas toujours existant)
```

**Points cles a retenir :**
- **Grounded** : Extension unique, position sceptique (la plus prudente)
- **Preferred** : Extensions maximales, positions credules (peut y en avoir plusieurs)
- **Stable** : N'existe pas toujours (cycles impairs), utiliser **CF2** dans ce cas
- Les cycles impairs posent probleme pour Stable -> utiliser CF2

## Prochaines etapes

Le notebook suivant explore l'**argumentation structuree** avec ASPIC+, DeLP, ABA et ASP.

---

**Navigation**: [<- Tweety-4-Belief-Revision](Tweety-4-Belief-Revision.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-6-Structured-Argumentation ->](Tweety-6-Structured-Argumentation.ipynb)